# Basic examples from the trimAl documentation

This example shows how to run the basic methods shown in the trimAl manual, but using the `pytrimal` API.

In [ ]:
import pytrimal
pytrimal.__version__

For this example, we will use one of the example alignments from the trimAl source repository. We use `Alignment.load` to load the alignment from a filename; note that `os.PathLike` objects are supported as well.

In [ ]:
import pathlib
ali = pytrimal.Alignment.load(pathlib.Path("data").joinpath("example.001.AA.clw"))

Let's see how the original alignment looks before trimming:

In [ ]:
for name, seq in zip(ali.names, ali.sequences):
    print(name.decode().ljust(10), seq)

## Example 1

Remove all positions in the alignment with gaps in 10% or more of the sequences, unless this leaves less than 60% of original alignment. In such case, print the 60% best (with less gaps) positions. Equivalent to:
```
$ trimal -in data/example.001.AA.clw -gt 0.9 -cons 60
```

In [ ]:
trimmer = pytrimal.ManualTrimmer(gap_threshold=0.9, conservation_percentage=60)
trimmed = trimmer.trim(ali)

for name, seq in zip(trimmed.names, trimmed.sequences):
    print(name.decode().ljust(10), seq)

## Example 2

Same as *Example 1*, but  the gap score is averaged over a window starting 3 positions before and ending 3 positions after each column.

In [ ]:
trimmer = pytrimal.ManualTrimmer(gap_threshold=0.9, conservation_percentage=60, window=3)
trimmed = trimmer.trim(ali)

for name, seq in zip(trimmed.names, trimmed.sequences):
    print(name.decode().ljust(10), seq)

## Example 3

Use an automatic method to decide optimal thresholds, based on the gap scores the input alignment. Equivalent to:
```
$ trimal -in data/example.001.AA.clw -gappyout
```

In [ ]:
trimmer = pytrimal.AutomaticTrimmer(method="gappyout")
trimmed = trimmer.trim(ali)

for name, seq in zip(trimmed.names, trimmed.sequences):
    print(name.decode().ljust(10), seq)

## Example 4

Use automatic methods to decide optimal thresholds, based on the combination of gap and similarity scores. Equivalent to:
```
$ trimal -in data/example.001.AA.clw -strictplus
```

In [ ]:
trimmer = pytrimal.AutomaticTrimmer(method="strictplus")
trimmed = trimmer.trim(ali)

for name, seq in zip(trimmed.names, trimmed.sequences):
    print(name.decode().ljust(10), seq)

## Example 5

Use an heuristic to decide the optimal method for trimming the alignment. Equivalent to:
```
$ trimal -in data/example.001.AA.clw -automated1
```

In [ ]:
trimmer = pytrimal.AutomaticTrimmer(method="automated1")
trimmed = trimmer.trim(ali)

for name, seq in zip(trimmed.names, trimmed.sequences):
    print(name.decode().ljust(10), seq)